In [19]:
import sys
sys.path

['',
 '/opencog/build/opencog/cython',
 '/opencog/opencog/python',
 '/opencog/opencog/nlp',
 '/usr/lib/python35.zip',
 '/usr/lib/python3.5',
 '/usr/lib/python3.5/plat-x86_64-linux-gnu',
 '/usr/lib/python3.5/lib-dynload',
 '/usr/local/lib/python3.5/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.5/dist-packages/IPython/extensions',
 '/home/opencog/.ipython',
 '/opencog/build/opencog/cython',
 '/opencog/opencog/python/',
 '/opencog/opencog/nlp/']

# Making some Atoms in python

Here we will mostly use the ipython shell to give examples. But you can of course put all the commands in a script_name.py script if you like and execute it, on terminal, using the command 

$ python script_name.py 

In [20]:
# Import AtomSpace and types to start making atoms.
from opencog.atomspace import AtomSpace, types

There are two ways to create Atoms in python. You can use the c++ like syntax or you can use the scheme like syntax.

# c++ like syntax

In [21]:
#declare an atomspace called atsp
atsp = AtomSpace()

In [22]:
#initializing atoms(nodes) of type ConceptNode.
cat = atsp.add_node(types.ConceptNode, "Cat")
animal = atsp.add_node(types.ConceptNode, "Animal")
man = atsp.add_node(types.ConceptNode, "Man")

In [24]:
#print a node
man

(ConceptNode "Man") ; [4943478229863008194][4]

Printing the AtomSpace will not quite work out as we might expect from c++. We can print its contents as follows:

In [25]:
#print the atomsapce
for atom in atsp:
    print (atom)

(ConceptNode "Man")

(ConceptNode "Animal")

(ConceptNode "Cat")



Now lets add some links into the AtomSpace. This is again, just like c++.

In [26]:
#initialize atom of type InheritanceLink.
atsp.add_link(types.InheritanceLink, [man, animal])

(InheritanceLink
  (ConceptNode "Man") ; [4943478229863008194][4]
  (ConceptNode "Animal") ; [1051227453015520217][4]
) ; [17186793166877131821][4]

# scheme like syntax

To use scheme like concise syntax you have to do a few extra steps. Import the type_constructors that enable you to add atoms to the AtomSpace by calling functions named after their types. Import initialize_opencog, a function that binds an AtomSpace instance (atsp in our case) to the type_constructors.

In [27]:
#Import initialize_opencog
from opencog.utilities import initialize_opencog

In [28]:
#Import type_constructors
from opencog.type_constructors import *

Now you can add atoms just like in scheme. This becomes very useful when you start building complex patterns for the pattern matcher.

In [29]:
#bind atsp with the type_constructors
initialize_opencog(atsp)

In [30]:
#initialize and add more atoms into atsp
color = ConceptNode("Color")
c1 = ConceptNode("Blue")
link = InheritanceLink(c1, color)

In [31]:
#print the atomspace
for atom in atsp:
    print (atom)

(ConceptNode "Cat")

(ConceptNode "Blue")

(ConceptNode "Animal")

(ConceptNode "Color")

(ConceptNode "Man")

(InheritanceLink
  (ConceptNode "Blue")
  (ConceptNode "Color")
)

(InheritanceLink
  (ConceptNode "Man")
  (ConceptNode "Animal")
)



One last thing before we wrap this up. The Atoms are classes with a lot of useful methods. If you are in the ipython shell you can look at a list of the methods by writing <variable_pointing_to_an_atom> < Tab >. For example, we have a variable called "color" that points to the ConceptNode object of name "Color". Then we can see a list of all the methods by:

color.< Press Tab >

# Pattern Matching

Now lets look at how we can run the PM through python. The PM functions are defined in c++ and made available in python through wrappers that are defined in atomspace/opencog/cython/opencog/bindlink.pyx and you can see the imports in atomspace/opencog/cython/opencog/bindlink.pxd

Here we will again run the equivalent of bindlink and cog-staisfying-set functions in scheme.

In [32]:
#Import the satisfying_set function
from opencog.bindlink import satisfying_set

In [33]:
#Delete existing atoms
atsp.clear()
#NOTE that the atoms are deleted only from atsp not from the entire python environment. 

In [34]:
#Create some atoms
InheritanceLink(ConceptNode("Red"), color)
InheritanceLink(ConceptNode("Green"), color)
InheritanceLink(ConceptNode("Blue"), color)

(InheritanceLink
  (ConceptNode "Blue") ; [3109859930626802082][4]
  (ConceptNode "Color") ; [8429230717376582988][4]
) ; [10172252740717271936][4]

In [35]:
#Create a pattern to look for color nodes
varlink = TypedVariableLink(VariableNode("$xcol"), TypeNode("ConceptNode"))
pattern = InheritanceLink(VariableNode("$xcol"), color)
colornodes = GetLink(varlink, pattern)

In [36]:
#Now run the function to find all color nodes
satisfying_set(atsp, colornodes)

(SetLink
  (ConceptNode "Blue") ; [3109859930626802082][4]
  (ConceptNode "Red") ; [6709574915108404932][4]
  (ConceptNode "Green") ; [7362795004147159697][4]
) ; [16291863344720241133][4]

In a similar manner you can import bindlink from opencog.bindlink and use it to execute rewrite queries (with the help of BindLink in place of SatisfactionLink)